## **Modelo de perceptron con funcion de activacion sigmoidal.**


Definicion de las clases que ocuparemos.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
class Node:
  def __call__(self, x):
    return self.forward(x)

  def __str__(self):
    return str(self.value)

#---------------------------------------------------------------------------------------------------
# Empezamos a crear nuestras sub clases que tambien seran nodos
# Primero definimos el que sera nuestro nodo principal para tres de nuestros modelo
# el cual es el nodo de pre-activacion
class PreActivation(Node):
    def __init__(self, input_size):
        self.weight = np.random.randn(input_size)
        self.bias = np.random.randn(1) # el bias es un tensor
        self.inputs = None

    def forward(self, x):
        self.inputs = x
        self.out = np.dot(x, self.weight) + self.bias
        return self.out


    # Grad_out es el gradiente calculado en el nodo sigmoide
    def backward(self, grad_out):
        self.grad_weight = grad_out * self.inputs
        self.grad_bias = np.sum(grad_out, axis=0)
        return self.grad_weight, self.grad_bias

    # Funcion escalonada del perceptron
    def predict(self,x):
        return np.where( self.forward(x)>0, 1, 0)
#---------------------------------------------------------------------------------------------------
# Definimos un nodo para la funcion de activacion Sigmoide
class Sigmoide(Node):
    def forward(self, z):
        self.out = 1 / (1 + np.exp(-z))
        return self.out

    # Grad_out es el gradiente calculado en el nodo CrossEntropy
    def backward(self, grad_out):
        grad_in = self.out * (1 - self.out)
        return grad_in * grad_out

    def predict(self,z):
        return np.where( self.forward(z)>0.5, 1, 0)

#---------------------------------------------------------------------------------------------------
# Definimos un nodo para la funcion de entropia cruzada
# La cual es necesaria para hacer el calculo del gradiente
class CrossEntropy(Node):
    def forward(self, y_pred, y_true):
        epsilon = 1e-15 # para evitar caer en un log(0)
        y_pred = np.clip(y_pred, epsilon, 1-epsilon)
        self.out = -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))
        return self.out
    def backward(self, y_pred, y_true):
        if y_true == 1:
            return -(1 / y_pred)
        else:
            if y_true == 0:
                return 1 / (1-y_pred)
        return self.out

Entrenamiento del modelo a partir de datos generados con la funcion "make_classification" de scikit-learn.

In [ ]:
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_informative=2, random_state=10)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# inicializamos nuestros nodos
np.random.seed(42)
# 1.- Nodo de pre-activacion
pre_activacion = PreActivation(2)
# 2.- Node de activacion (sigmoide)
sigmoide = Sigmoide()
# 3.- Nodo de entropia cruzada
entropia_cruzada = CrossEntropy()

# inicializamos nuestra variable para tasa de aprendizaje y epocas
lr = 0.01
epochs = 100

# paso forward para 1 iteracion
history = []
for epoch in range(epochs):
    history_for_epoch = []
    for i in range(len(y_train)):
        # paso forward
        z = pre_activacion.forward(x_train[i])
        y_pred = sigmoide.forward(z)
        loss = entropia_cruzada.forward(y_pred, y_train[i])

        # paso backward
        grad_loss = entropia_cruzada.backward(y_pred, y_train[i])
        grad_sigmoid = sigmoide.backward(grad_loss)
        grad_weights, grad_bias = pre_activacion.backward(grad_sigmoid)

        # actualizacion de los pesos
        pre_activacion.weight -= lr * grad_weights
        pre_activacion.bias -= lr * grad_bias
        history_for_epoch.append(loss)
    history.append(np.mean(history_for_epoch))
    print(f'Epoch: {epoch + 1}, loss: {loss}')

Resultados del entrenamiento

In [ ]:
from sklearn.metrics import classification_report
z = pre_activacion.forward(x_test)
y_pred_test = sigmoide.predict(z)
report = classification_report(y_test, y_pred_test)
print(report)

Grafica que modela el comportamiento de la perdida promedio a lo largo de las epocas.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(list(range(epochs)), history)
plt.title('Comportamiento de la Pérdida promedio del modelo a lo largo de las épocas')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.show()

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_pred_test)
plt.title('Datos clasificados')
plt.xlabel('Caracteristiica 1')
plt.ylabel('Caracteristica 2')
plt.show("hpoa")


Comparacion de la funcion escalonada con la regresion logistica.

In [ ]:
print(y_pred_test)
y_escalonada = pre_activacion.predict(x_test)
print(y_escalonada)

## **Modelo de perceptron con una funcion de regresion lineal como funcion de activacion:**


Definicion de las clases que ocuparemos.

In [ ]:
#---------------------------------------------------------------------------------------------------
# Empezamos a crear nuestras sub clases que tambien seran nodos
# Primero definimos el que sera nuestro nodo principal para tres de nuestros modelo
# el cual es el nodo de pre-activacion
class PreActivation(Node):
    def __init__(self, input_size):
        self.weight = np.random.randn(input_size) # el peso o los pesos son un tensor
        self.bias = np.random.randn(1) # el bias es un tensor
        self.inputs = None

    def forward(self, x):
        self.inputs = x
        self.out = np.dot(x, self.weight) + self.bias
        return self.out

    # aqui el grad_out es el gradiente calculado en el nodo sigmoide
    def backward(self, grad_out):
        #self.grad_weight = np.dot(self.inputs.T, grad_out)
        self.grad_weight = grad_out * self.weight
        self.grad_bias = np.sum(grad_out, axis=0)
        return self.grad_weight, self.grad_bias


#---------------------------------------------------------------------------------------------------
# Definimos un nodo para la funcion de entropia cruzada
# La cual es necesaria para hacer el calculo del gradiente
class mean_square_error(Node):
    def forward(self, y_pred, y_true):
        return mean_squared_error(y_true, y_pred)

    def backward(self, y_pred, y_true):
        #return 2*(y_true-y_pred)
        return (y_pred-y_true)

class MeanSquareError(Node):
    def forward(self, y_pred, y_true):
        self.out = np.mean((y_pred - y_true.reshape(-1,1)) ** 2)
        #self.out = mean_squared_error(y_true, y_pred)
        return self.out

    def backward(self, y_pred, y_true):
        self.grad = (2 * (y_pred - y_true.reshape(-1,1))) / y_true.size
        return self.grad

Entrenamiento del modelo.

In [ ]:
# inicializamos nuestros nodos
np.random.seed(42)
# 1.- Nodo de pre-activacion
pre_activacion = PreActivation(input_size = x_train.shape[1])
# 2.- Node de mean square error
objetive_funtion = mean_square_error()

# inicializamos nuestra variable para tasa de aprendizaje y epocas
lr = 0.01
epochs = 200

# paso forward para 1 iteracion
history = []
for epoch in range(epochs):
    history_for_epoch = []
    # paso forward
    y_pred = pre_activacion.forward(x_train)
    loss = objetive_funtion.forward(y_pred, y_train)

    # paso backward
    grad_loss = np.mean(objetive_funtion.backward(y_pred, y_train))
    grad_weights, grad_bias = pre_activacion.backward(grad_loss)

    # actualizacion de los pesos
    pre_activacion.weight -= lr * grad_weights
    pre_activacion.bias -= lr * grad_bias
    history.append(loss)
    print(f'Epoch: {epoch + 1}, loss: {loss}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(list(range(epochs)), history)
plt.title('Comportamiento de la Pérdida del Modelo a lo Largo de las Épocas')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.show()

Evaluacion del modelo.

In [ ]:

y_pred_test = pre_activacion.forward(x_train)
MSE = mean_squared_error(y_train, y_pred_test)
r2 = r2_score(y_train, y_pred_test)
print("El MSE es: ", MSE)
print("El R2 es: ", r2)